# Naive Bayes Classifier

In [3]:
import os

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import numpy as np
import torch

In [24]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
dtype = torch.float16

n_samples = 1000
n_features = 10
n_classes = 5

X_train = torch.tensor(
    np.random.rand(n_samples, n_features), dtype=dtype, device=device
)
y_train = torch.randint(
    low=0, high=n_classes, size=(n_samples,), device=device, dtype=dtype
)

X_test = torch.tensor(np.random.rand(10, n_features), dtype=dtype, device=device)

In [61]:
class NaiveBayesClassifer_LL1:
    def fit(self, X: torch.Tensor, y: torch.Tensor):
        n_samples, n_features = X.size()
        self.classes = torch.unique(y)
        self.n_classes = self.classes.size(0)

        self._mean = torch.zeros(
            (self.n_classes, n_features), dtype=dtype, device=device
        )
        self._var = torch.zeros(
            (self.n_classes, n_features), dtype=dtype, device=device
        )
        self._priors = torch.zeros(self.n_classes, dtype=dtype, device=device)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self._mean[idx, :] = torch.mean(X_c, dim=0)
            self._var[idx, :] = torch.var(X_c, dim=0)
            self._priors[idx] = X_c.size(0) / n_samples

    def predict(self, X):
        y_preds = []
        for x in X:
            logp = self.get_logp(x)
            pred_c = torch.argmax(torch.tensor(logp))
            y_preds.append(pred_c)
        return y_preds

    def get_logp(self, x):
        posteriors = []
        for idx, c in enumerate(self.classes):
            prior = torch.log(self._priors[idx])
            class_conditional = torch.sum(torch.log(self.pdf(idx, x)))
            posteriors.append(prior + class_conditional)
        return posteriors

    def pdf(self, c: int, x: torch.Tensor):
        mean = self._mean[c]
        var = self._var[c]
        numerator = torch.exp(-(x - mean) ** 2 / (2 * var))
        denominator = torch.sqrt(2 * torch.pi * var)
        return numerator / denominator


classifier = NaiveBayesClassifer_LL1()
classifier.fit(X, y)
classifier.predict(X_test)

[tensor(3),
 tensor(1),
 tensor(2),
 tensor(1),
 tensor(2),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(3),
 tensor(1)]

In [60]:
class NBC_LL2:
    def fit(self, X: torch.Tensor, y: torch.Tensor):
        n_samples, n_features = X.size()
        self.classes = torch.unique(y)
        self.n_classes = self.classes.size(0)

        self.mean = torch.zeros(
            (self.n_classes, n_features), device=device, dtype=dtype
        )
        self.var = torch.zeros((self.n_classes, n_features), device=device, dtype=dtype)
        self.priors = torch.zeros(self.n_classes, device=device, dtype=dtype)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(dim=0)
            self.var[idx, :] = X_c.var(dim=0)
            self.priors[idx] = X_c.size(0) / X.size(0)

    def logp(self, X: torch.Tensor):
        X = X.unsqueeze(1)
        log_ccp = -0.5 * torch.sum(
            torch.log(2 * torch.pi * self.var) + (X - self.mean) ** 2 / self.var, dim=-1
        )
        log_prior = torch.log(self.priors)

        log_posterior = log_ccp + log_prior

        return log_posterior

    def predict(self, X: torch.tensor):
        logp = self.logp(X)
        predictions = torch.argmax(logp, dim=1)
        return predictions


classifier = NBC_LL2()
classifier.fit(X, y)
classifier.predict(X_test)

tensor([3, 1, 2, 1, 2, 0, 1, 0, 3, 1], device='mps:0')

In [94]:
class NBC_LL29:
    def fit(self, X, y):
        n_samples, n_features = X.size()
        self.classes = torch.unique(y)
        self.n_classes = self.classes.size(0)
        self.mean = torch.zeros(self.n_classes, n_features, dtype=dtype, device=device)
        self.var = torch.zeros(self.n_classes, n_features, dtype=dtype, device=device)
        self.priors = torch.zeros(self.n_classes, dtype=dtype, device=device)
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(dim=0)
            self.var[idx, :] = X_c.var(dim=0)
            self.priors[idx] = X_c.size(0) / X.size(0)

    def logp(self, X):
        X = X.unsqueeze(1)

        log_ccp = -0.5 * (
            torch.sum(
                torch.log(2 * torch.pi * self.var) + ((X - self.mean) ** 2 / self.var),
                dim=-1,
            )
        )
        log_prior = torch.log(self.priors)

        log_posterior = log_prior + log_ccp

        return log_posterior

    def predict(self, X):
        logp = self.logp(X)
        return torch.argmax(logp, dim=-1)


classifier = NBC_LL29()
classifier.fit(X, y)
classifier.logp(X_test)
classifier.predict(X_test)

tensor([3, 1, 2, 1, 2, 0, 1, 0, 3, 1], device='mps:0')

In [104]:
class NBC_LL295:
    def fit(self, X, y):
        n_samples, n_features = X.size()
        self.classes = torch.unique(y)
        self.n_classes = self.classes.size(0)

        self.mean = torch.zeros(self.n_classes, n_features, dtype=dtype, device=device)
        self.var = torch.zeros(self.n_classes, n_features, dtype=dtype, device=device)
        self.priors = torch.zeros(self.n_classes, dtype=dtype, device=device)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(dim=0)
            self.var[idx, :] = X_c.var(dim=0)
            self.priors[idx] = X_c.size(0) / X.size(0)

    def logp(self, X):
        X = X.unsqueeze(1)
        log_ccp = -0.5 * (
            torch.sum(
                torch.log(2 * torch.pi * self.var) + ((X - self.mean) ** 2 / self.var),
                dim=-1,
            )
        )

        log_prior = torch.log(self.priors)

        return log_ccp + log_prior

    def predict(self, X):
        logp = self.logp(X)
        return torch.argmax(logp, dim=-1)


classifier = NBC_LL295()
classifier.fit(X, y)
classifier.logp(X_test)
classifier.predict(X_test)

tensor([3, 1, 2, 1, 2, 0, 1, 0, 3, 1], device='mps:0')

In [110]:
class NBC_LL3:
    def fit(self, X, y):
        n_samples, n_features = X.size()
        self.classes = torch.unique(y)
        self.n_classes = self.classes.size(0)

        self.mean = torch.zeros(self.n_classes, n_features, dtype=dtype, device=device)
        self.var = torch.zeros(self.n_classes, n_features, dtype=dtype, device=device)
        self.priors = torch.zeros(self.n_classes, dtype=dtype, device=device)
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(dim=0)
            self.var[idx, :] = X_c.var(dim=0)
            self.priors[idx] = X_c.size(0) / X.size(0)

    def logp(self, X):
        X = X.unsqueeze(1)
        log_ccp = -0.5 * (
            torch.sum(
                torch.log(2 * torch.pi * self.var) + (X - self.mean) ** 2 / self.var,
                dim=-1,
            )
        )
        log_prior = torch.log(self.priors)
        return log_prior + log_ccp
        
    def predict(self, X):
        logp = self.logp(X)
        return torch.argmax(logp, dim=-1)

classifier = NBC_LL3()
classifier.fit(X, y)
classifier.logp(X_test)
classifier.predict(X_test)

tensor([3, 1, 2, 1, 2, 0, 1, 0, 3, 1], device='mps:0')